In [16]:
import pandas as pd 
import polars as pl
import sys 
import os 
import numpy as np 
sys.path.append("..")

from settings import NB_TRANSACTIONS_PER_MONTH, PROJECT_PATH, REGRESSION_TARGET,CLASSIFICATION_TARGET

In [17]:
transactions = pl.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_post_feature_engineering.parquet")
)

Comme exemple de modèle étalon un peu plus complexe, on peut généraliser l'idée d'avoir la moyenne de la target en calculant cette fois-ci la moyenne de la ville pour chaque mois ! Et c'est une information que nous avons déjà calculé d'ailleurs pendant la phase de feature engineering ! L'information est stockée dans la colonne prix_m2_moyen_mois_precedent

In [18]:
transactions["prix_m2_moyen_mois_precedent"]

prix_m2_moyen_mois_precedent
f64
1241.194799
1241.194799
1241.194799
1241.194799
1241.194799
…
3853.821448
3853.821448
3711.043574


La "prediction" du "modèle" devient alors tout simplement la valeur de cette colonne

In [19]:
X = transactions.drop([REGRESSION_TARGET, CLASSIFICATION_TARGET])
y_pred = transactions["prix_m2_moyen_mois_precedent"]
y_regression = transactions[REGRESSION_TARGET]

Nous avons désormais des "predictions" ainsi que notre target. On peut alors calculer toutes nos métriques de performances comme nous le souhaitons ! 

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [22]:

rmse = np.sqrt(mean_squared_error(y_pred, y_regression))
print("RMSE : ", rmse)
mae = mean_absolute_error(y_pred, y_regression)
print("MAE : ", mae)
r2 = r2_score(y_pred, y_regression)
print("R2 : ", r2)

RMSE :  371474.18203455745
MAE :  242045.18741309742
R2 :  -13640.792490333895


Un R2 négatif ! Il y a clairement espoir qu'un modèle de ML puisse faire mieux que cela ! Ce résultat est aussi intéressant car il nous permets de déduire qu'il n'est pas réaliste d'obtenir une estimation fiable du prix d'une transaction en se basant uniquement sur une moyenne globale.

Remarquez qu'ici, nous n'avons pas réalisé de séparation train et test. Cela serait en effet inutile car nous n'utilisons pas un modèle statistique, mais une règle déterministe qui ne connait pas l'overfit ou l'underfit  ! 